# 利用ground truth,制作点特征的label

移动点云数据

In [4]:
import shutil
import os


with open('../datasets/paths.txt', 'r') as file:
    lines = file.readlines()

for line in lines:
    parts = line.strip().split(' ', 1)  # 使用第一个空格来分割
    if len(parts) == 2:  # 确保行正确分割为两部分
        index, path = parts
        # print(f"序号: {index}, 路径: {path}")
        source_path = os.path.join(path, 'velodyne_points/data')
        target_path = os.path.join('../datasets/sequence_point', f"{index}")
        target_path = os.path.join(target_path, 'velodyne')
        shutil.copytree(source_path, target_path)
        print(f"成功复制：{source_path} 到 {target_path}")


成功复制：../datasets/2011_10_03/2011_10_03_drive_0027_sync/velodyne_points/data 到 ../datasets/sequence_point/00/velodyne
成功复制：../datasets/2011_10_03/2011_10_03_drive_0042_sync/velodyne_points/data 到 ../datasets/sequence_point/01/velodyne
成功复制：../datasets/2011_10_03/2011_10_03_drive_0034_sync/velodyne_points/data 到 ../datasets/sequence_point/02/velodyne
成功复制：../datasets/2011_09_30/2011_09_30_drive_0016_sync/velodyne_points/data 到 ../datasets/sequence_point/04/velodyne
成功复制：../datasets/2011_09_30/2011_09_30_drive_0018_sync/velodyne_points/data 到 ../datasets/sequence_point/05/velodyne
成功复制：../datasets/2011_09_30/2011_09_30_drive_0020_sync/velodyne_points/data 到 ../datasets/sequence_point/06/velodyne
成功复制：../datasets/2011_09_30/2011_09_30_drive_0027_sync/velodyne_points/data 到 ../datasets/sequence_point/07/velodyne
成功复制：../datasets/2011_09_30/2011_09_30_drive_0033_sync/velodyne_points/data 到 ../datasets/sequence_point/09/velodyne
成功复制：../datasets/2011_09_30/2011_09_30_drive_0034_sync/velodyne_

读取每个GT，将GT转换成velo to velo

In [8]:
import numpy as np

In [9]:
def read_calib_file(filepath):
    calib_data = {}
    with open(filepath, 'r') as file:
        for line in file:
            key, value = line.split(':', 1)
            if key == 'R' or key == 'T':
                calib_data[key] = value.strip()

    return calib_data


def RT_matrix_maker(calib_data):
    if "R" in calib_data and "T" in calib_data:
        R_values = [float(x) for x in calib_data["R"].split()]
        T_values = [float(x) for x in calib_data["T"].split()]

        # 构造变换矩阵
        R_matrix = np.array(R_values).reshape(3, 3)
        T_vector = np.array(T_values).reshape(3, 1)
        RT_matrix = np.hstack((R_matrix, T_vector))
        RT_matrix = np.vstack((RT_matrix, [0, 0, 0, 1]))

        RT_matrix
    else:
        RT_matrix = None

    return RT_matrix

In [20]:
with open('../datasets/paths.txt', 'r') as file:
    lines = file.readlines()

RT_matrix = np.eye(4)
for line in lines:
    parts = line.strip().split(' ', 1)  # 使用第一个空格来分割
    if len(parts) == 2:  # 确保行正确分割为两部分
        index, path = parts
        calib_filepath = os.path.join(os.path.dirname(path), 'calib_velo_to_cam.txt')
        calib_data = read_calib_file(calib_filepath)
        RT_matrix = RT_matrix_maker(calib_data)
        RT_matrix_inv = np.linalg.inv(RT_matrix)
        gt_filepath = os.path.join(path, '{}.txt'.format(index))

    with open(gt_filepath, 'r') as f:
        lines = f.readlines()

    transformed_lines = []

    for line in lines:
        pose = line.strip().split()  # 切分每行数据

        # 提取位姿变换矩阵并变为4x4
        pose_matrix = np.eye(4)
        
        pose_matrix[:3, :] = np.array(pose).reshape(3, 4).astype(float)

        # 旋转矩阵
        transformed_matrix = RT_matrix_inv @ pose_matrix @ RT_matrix

        # 从变换矩阵中提取变换后的位姿
        transformed_pose = transformed_matrix[:3, :].flatten()
        transformed_lines.append(" ".join(map(str, transformed_pose)))
    
    data_matrices = []
    for line in transformed_lines:
        pose = line.strip().split()  # 切分每行数据
        data_matrix = np.eye(4)
        data_matrix[:3, :] = np.array(pose).reshape(3, 4).astype(float)
        data_matrices.append(data_matrix)

    data_matrices = np.array(data_matrices)
    
    T_filename = os.path.join('../datasets/sequence_point/{}'.format(index), 'T')
    os.mkdir(T_filename)
    for i in range(data_matrices.shape[0] - 1):
        T = np.dot(np.linalg.inv(data_matrices[i+1]), data_matrices[i])
        np.savetxt(os.path.join(T_filename, '{:010d}.txt'.format(i)), T)
    #T 是从 i+1 到 i 的变换矩阵
        



        

